In [1]:
from langchain.vectorstores import Milvus
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

True

In [2]:
df = pd.read_csv('../Data/preguntas_noticias_santiago.csv')

In [3]:
df

,q,true answer,ft-base,correct ans,Precision,False Answers,True Answers,Null Answers
0,"A inicios de enero de 2023, la ministra de Jus...",Verdadero,-,False,"0,3846153846","0,6538461538","0,2179487179","0,1282051282"
1,El ministro de Justicia Luis Cordero asume su ...,Verdadero,Falso,False,Prompt,NaN,NaN,NaN
2,El presidente Boric indulto a 12 condenados po...,Verdadero,Falso,False,¿Es verdad que {news}?,NaN,NaN,NaN
3,Marcela Ríos se libró de una acusación constit...,Verdadero,Falso,False,NaN,NaN,NaN,NaN
4,Gabriel Boric Font es un político Chileno que ...,Verdadero,Falso,False,NaN,Precision F,NaN,NaN
...,...,...,...,...,...,...,...,...
73,La comisión de expertos tiene un año para elab...,Falso,Verdadero,False,NaN,NaN,NaN,NaN
74,Claudio Grossman es un miembro del consejo con...,Verdadero,Falso,False,NaN,NaN,NaN,NaN
75,Estefanía Esparza (Partido Radical) es una mie...,Verdadero,Falso,False,NaN,NaN,NaN,NaN
76,María Rojas es una miembra del consejo constit...,Falso,Falso,True,NaN,NaN,NaN,NaN


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [11]:
vector_db = Milvus(
    embeddings,
    connection_args={"uri":os.getenv("MILVUS_ENDPOINT"), "token": os.getenv("MILVUS_API_KEY")},
    collection_name="News_2023",
    text_field="body" # Set this to the name of your text field
)

In [32]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_together import TogetherEmbeddings
from langchain_community.llms import Together

retriever = vector_db.as_retriever()

model = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0,
    max_tokens=20,
    top_k=50,
    together_api_key=os.getenv("TOGETHER_API_KEY")
)

# Provide a template following the LLM's original chat template.
template = """<s>[INST] Responde verdadero, falso o no lo se la siguiente afirmacion, usando el siguiente contexto:
{context}
Solo responde con verdadero, falso o no lo se. Nunca des una explicación a tu respuesta.

Afirmacion: {afirmacion} [/INST] 
"""
prompt = ChatPromptTemplate.from_template(template) 

# chain = (
#     {"context": (retriever | (lambda docs: print([d.page_content for d in docs]))), "afirmacion": RunnablePassthrough()}
#     | prompt
#     | model
#     | StrOutputParser()
# )

def get_retrieved_docs(docs):
    return [d.page_content for d in docs]

chain = (
    {"context": retriever | get_retrieved_docs,
     "afirmacion": RunnablePassthrough(),
     "retrieved_docs": retriever | get_retrieved_docs}
    | prompt
    | model
    | StrOutputParser()
)

In [10]:
docs = vector_db.similarity_search(df['q'][0])

In [20]:
output = chain.invoke('A inicios de enero de 2023, la ministra de Justicia Marcela Ríos renuncia a su cargo por presiones opositoras')

In [26]:
retriever.get_relevant_documents('A inicios de enero de 2023, la ministra de Justicia Marcela Ríos renuncia a su cargo por presiones opositoras')

[Document(page_content='Marcela Ríos deja su cargo como ministra de Justicia Luis Cordero Vega asumirá como nuevo titular de la cartera de Justicia, según explicó el Presidente Gabriel Boric. El Gobierno confirmó este sábado 7 de enero que Marcela Ríos renunció a su cargo como ministra de Justicia. El Presidente de la República, Gabriel Boric, explicó que "debido a que hubo desprolijidades en la ejecución de mi decisión de conceder indultos y considerando además la necesidad de fortalecer la gestión política del Ministerio de Justicia y Derechos Humanos, he decidido aceptar la renuncia de Marcela Ríos a dicha cartera', metadata={'Auto_id': 448075995546842535, 'title': 'Marcela Ríos deja su cargo como ministra de Justicia', 'uri': '7338639415', 'url': 'https://www.24horas.cl/actualidad/politica/marcela-rios-deja-su-cargo-como-ministra-de-justicia', 'dateTimePub': '2023-01-07 19:41:00+00:00', 'dataType': 'news'}),
 Document(page_content='Marcela Ríos renuncia al Ministerio de Justicia tr

In [33]:
import csv
import time
# Abrir el archivo CSV en modo añadir
with open('../Data/resultado3_rag.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    
    # Escribir los encabezados de las columnas (si el archivo está vacío)
    if file.tell() == 0:
        writer.writerow(['question','True','Predicted', 'docs'])
    
    # Iterar sobre el DataFrame original
    for index, row in df.iterrows():
        question = row['q']
        true_ans = row['true answer']
        output = chain.invoke(question)
        docs = retriever.get_relevant_documents(question)
        time.sleep(1)
        
        # Escribir los valores en el archivo CSV
        writer.writerow([question,true_ans, output, docs])


In [4]:
import pandas as pd

df2 = pd.read_csv('../Data/resultado3_rag.csv')
j = 0
for i,row in df2.iterrows():
    if row[1].lower() == row[2].lower():
        j+=1

j

/var/folders/g_/lrzwrr894rn133_2g0g859y00000gn/T/ipykernel_8008/1745383157.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1].lower() == row[2].lower():


44

In [3]:
import pandas as pd
df3 = pd.read_csv('../Data/respuestas_preguntas_noticias_santiago_base_model.csv')
j2 = 0
for i,row in df3.iterrows():
    if row[1].lower() == row[2].lower():
        j2+=1

j2

/var/folders/g_/lrzwrr894rn133_2g0g859y00000gn/T/ipykernel_8008/1378620086.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1].lower() == row[2].lower():


41

# Documentos del Retriever

In [7]:
for i, row in df2.iterrows():
    if row[2].lower() != row[1].lower():
        print(row[0])
        for j in row['docs']:
            print(j)

/var/folders/g_/lrzwrr894rn133_2g0g859y00000gn/T/ipykernel_8008/1618850329.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[2].lower() != row[1].lower():
/var/folders/g_/lrzwrr894rn133_2g0g859y00000gn/T/ipykernel_8008/1618850329.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])


El ministro de Justicia Luis Cordero asume su cargo en enero del 2023
[
D
o
c
u
m
e
n
t
(
p
a
g
e
_
c
o
n
t
e
n
t
=
'
S
i
n
 
c
e
r
e
m
o
n
i
a
 
a
s
u
m
i
r
á
 
h
o
y
 
e
l
 
n
u
e
v
o
 
m
i
n
i
s
t
r
o
 
d
e
 
J
u
s
t
i
c
i
a
 
L
u
i
s
 
C
o
r
d
e
r
o
 
S
i
n
 
c
e
r
e
m
o
n
i
a
 
a
s
u
m
i
r
á
 
h
o
y
 
e
l
 
n
u
e
v
o
 
m
i
n
i
s
t
r
o
 
d
e
 
J
u
s
t
i
c
i
a
 
L
u
i
s
 
C
o
r
d
e
r
o
,
 
q
u
i
e
n
 
f
u
e
 
n
o
m
b
r
a
d
o
 
p
o
r
 
e
l
 
P
r
e
s
i
d
e
n
t
e
 
G
a
b
r
i
e
l
 
B
o
r
i
c
 
e
n
 
m
e
d
i
o
 
d
e
 
l
a
 
p
o
l
é
m
i
c
a
 
d
e
l
 
i
n
d
u
l
t
o
,
 
y
 
l
u
e
g
o
 
d
e
 
l
a
 
s
a
l
i
d
a
 
d
e
 
l
a
 
a
h
o
r
a
 
e
x
j
e
f
a
 
d
e
 
l
a
 
c
a
r
t
e
r
a
 
M
a
r
c
e
l
a
 
R
í
o
s
.
 
L
a
 
o
p
o
s
i
c
i
ó
n
 
e
n
 
t
a
n
t
o
 
a
n
u
n
c
i
ó
 
l
a
 
c
r
e
a
c
i
ó
n
 
d
e
 
u
n
a
 
c
o
m
i
s
i
ó
n
 
e
s
p
e
c
i
a
l
 
p
a
r
a
 
i
n
v
e
s
t
i
g
a
r
 
l
a
 
e
n
t
r
e
g
a
 
d
e
 
l
o
s
 
1
3
 
i
n
d
u
l
t
o
s
 
o
t
o
r
g
a
d
o
s
 
p
o
r
 
e
l
 
m
a
n
d
a
t
a
r
i
o
.
 
A
n
t
i
